In [433]:
file_data = '9.txt'
data = []
with open(file_data) as f:
    for line in f:
        data.append([float(x) for x in line.split()])
data = list(filter(None, data))
max_weight = data[0][0]
max_size = data[0][1]

del data[0]

In [434]:
import random

### Случайная генерация

In [435]:
#def random_generation():
#    simple_list = [0,1]
#    random_generation = []
#    for i in range(200):
#        random_generation.append(random.choices(simple_list, k=30))
#    return random_generation

### Выбор 20% особей для скрещивания

In [436]:
def fitness_func(percent, random_generation):
    fitness = []
    for i in range(len(random_generation)):
        sum_weight, sum_size, price = 0,0,0
        for k in range(30):
            sum_weight += random_generation[i][k] * data[k][0]
            sum_size += random_generation[i][k] * data[k][1]
            price += random_generation[i][k]*data[k][2]
            k = k+1
        if (sum_weight<=max_weight) and (sum_size<=max_size):
            fitness.append(price)
        else:
            fitness.append(0)
        i = i+1
    fitness_sort = sorted(fitness, reverse=True)
    fitness_sort_percent = []
    for i in range(int(percent*len(fitness_sort))):
        fitness_sort_percent.append(fitness_sort[i])
    random_generation_percent = []
    for i in range(len(fitness)):
        if (fitness[i] in fitness_sort_percent) and (fitness[i]>0):
            random_generation_percent.append(random_generation[i])
        i = i+1
    return random_generation_percent

### Многоточечное скрещивание

In [437]:
def crossing(random_generation_twenty):
    descendants = []
    for i in range(0, len(random_generation_twenty)-1, 2):
        descendant1 = []
        descendant2 = []
        descendant1.extend(random_generation_twenty[i][0:10])
        descendant1.extend(random_generation_twenty[i+1][10:20])
        descendant1.extend(random_generation_twenty[i][20:30])
        descendant2.extend(random_generation_twenty[i+1][0:10])
        descendant2.extend(random_generation_twenty[i][10:20])
        descendant2.extend(random_generation_twenty[i+1][20:30])
        descendants.append(descendant1)
        descendants.append(descendant2)
    return descendants

### Мутация: добавление 1 случайной вещи 5% особей

In [438]:
def mutation(descendants):
    mutated_descendants = descendants
    for i in range(int(0.05*len(mutated_descendants))):
        random_bit = random.randint(0,29)
        if mutated_descendants[i][random_bit] == 0:
                mutated_descendants[i][random_bit] = 1
#         else:
#             while (mutated_descendants[i][random_bit] != 0) and (random_bit<30):
#                 random_bit = random_bit+1
#             mutated_descendants[i][random_bit] = 1
    return mutated_descendants

###  Замена 20% худших особей из предыдущего поколения на лучших потомков 

In [439]:
def formation_new_population(random_generation_twenty, mutated_descendants):
    random_generation_eighty_best = fitness_func(0.8, random_generation_twenty)
    descendants_twenty_best = fitness_func(0.2, mutated_descendants)
    new_population = []
    new_population = random_generation_eighty_best + descendants_twenty_best
    return new_population

In [440]:
def fitness_func_best_individual(new_population):
    best_sum_weight = 0 
    best_sum_size = 0
    best_price = 0
    best_collection = []
    for i in range(len(new_population)):
        sum_weight = 0 
        sum_size = 0
        price = 0
        for k in range(30):
            sum_weight += new_population[i][k] * data[k][0]
            sum_size += new_population[i][k] * data[k][1]
            price += new_population[i][k]*data[k][2]
            k = k+1
        if (sum_weight<=max_weight) and (sum_size<=max_size) and (price>best_price):
            best_price = price
            best_sum_weight = sum_weight
            best_sum_size = sum_size
            best_collection = new_population[i]
        i = i+1
    return best_collection

In [441]:
def calculation_price_best_collection(best_collection):
    price_best_collection = 0
    for k in range(30):
        price_best_collection += best_collection[k]*data[k][2]
        k = k+1
    return price_best_collection

In [466]:
old_price_best_collection = 0
new_price_best_collection = 0
c = 500
min_price = 0
all_price = []
while (c>0) and abs(old_price_best_collection - new_price_best_collection) >= min_price:
    for i in range (len(data)):
        all_price.append(data[i][2])
    min_price = min(all_price)
    old_price_best_collection = new_price_best_collection
    simple_list = [0,1]
    random_generation = []
    for i in range(200):
        random_generation.append(random.choices(simple_list, k=30))
    random_generation_twenty = fitness_func(0.2, random_generation)
    descendants = crossing(random_generation_twenty)
    mutated_descendants = mutation(descendants)
    new_population = formation_new_population(random_generation_twenty, mutated_descendants)
    best_collection = fitness_func_best_individual(new_population)
    new_price_best_collection = calculation_price_best_collection(best_collection)
    c -= 1
sum_weight, sum_size, sum_price = 0,0,0
res = []
for i in range (len(best_collection)):
    sum_weight += best_collection[i] * data[i][0]
    sum_size += best_collection[i] * data[i][1]
    sum_price += best_collection[i] * data[i][2]
    if best_collection[i] == 1:
        res.append([i, data[i][0], data[i][1], data[i][2]])

In [468]:
print("Грузоподъёмность рюкзака:", max_weight, "\nВес сложенного груза:", sum_weight, 
"\nВместимость рюкзака:", max_size, "\nОбъем сложенного груза:", sum_size, 
"\nЦенность сложенного груза:", sum_price)
res

Грузоподъёмность рюкзака: 13000.0 
Вес сложенного груза: 12743.0 
Вместимость рюкзака: 12.0 
Объем сложенного груза: 8.9 
Ценность сложенного груза: 3821.0


[[0, 816.0, 0.4, 356.0],
 [1, 1236.0, 0.5, 202.0],
 [2, 1391.0, 0.5, 294.0],
 [4, 475.0, 0.9, 225.0],
 [7, 1526.0, 0.6, 356.0],
 [10, 1518.0, 0.8, 369.0],
 [12, 1216.0, 0.6, 251.0],
 [16, 149.0, 0.5, 370.0],
 [18, 607.0, 0.7, 359.0],
 [20, 950.0, 0.6, 207.0],
 [21, 1342.0, 1.1, 317.0],
 [27, 128.0, 0.9, 384.0],
 [28, 1389.0, 0.8, 131.0]]